<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries-needed" data-toc-modified-id="Libraries-needed-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries needed</a></span></li><li><span><a href="#Request-data-from-de-API" data-toc-modified-id="Request-data-from-de-API-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Request data from de API</a></span><ul class="toc-item"><li><span><a href="#To-request-data-from-the-server:" data-toc-modified-id="To-request-data-from-the-server:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>To request data from the server:</a></span></li><li><span><a href="#To-get-the-total-number-of-occurrences-for-each-species" data-toc-modified-id="To-get-the-total-number-of-occurrences-for-each-species-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>To get the total number of occurrences for each species</a></span></li></ul></li></ul></div>

### Libraries needed

pygbif is split up into modules for each of the major groups of API methods.

Registry - Datasets, Nodes, Installations, Networks, Organizations
Species - Taxonomic names
Occurrences - Occurrence data, including the download API
Maps - Maps, get raster maps from GBIF as png or mvt

In [1]:
import pygbif
from pygbif import occurrences as occ
from pygbif import species

import requests

import pandas as pd

### Request data from de API

#### To request data from the server:

In [2]:
url = 'http://api.gbif.org/v1/species/match?verbose=true&name='
data = requests.get(url)
data.status_code

200

In order to call the species with the GBIF API, it is necessary to enter the codes for each species. As it is impossible to know the codes, I am going to generate a function that, giving the scientific name, will return the codes for each one of them.

In [53]:
splist = ['Apus affinis', 'Apus caffer', 'Parnasius apollo', 'Calotriton arnoldi', 'Gallotia simonyi', 'Alytes dickhilleni',
            'Meliataea cinxia', 'Silene ciliata', 'Abies pinsapo', 'Lanius minor', 'Ciconia ciconia', 'Sonchus tenerrimus', 
          'Araujia sericifera']

def get_species_name_from_codes (sp):
    keys = [species.name_backbone(x)['usageKey'] for x in splist ]
    species_codes = dict(zip(splist, keys))
    return species_codes    

In [54]:
code_number = get_species_name_from_codes(splist)
code_number

{'Apus affinis': 5228662,
 'Apus caffer': 5228644,
 'Parnasius apollo': 1938810,
 'Calotriton arnoldi': 2431827,
 'Gallotia simonyi': 2468673,
 'Alytes dickhilleni': 2426613,
 'Meliataea cinxia': 8168644,
 'Silene ciliata': 7267056,
 'Abies pinsapo': 2685464,
 'Lanius minor': 2492854,
 'Ciconia ciconia': 2481912,
 'Sonchus tenerrimus': 3105768,
 'Araujia sericifera': 3170442}

#### To get the total number of occurrences for each species
We can use the function occ.search which search for GBIF occurences.

In [5]:
response = requests.get('http://api.gbif.org/v1/occurrence/counts/datasets')
response.status_code

200

In [6]:
code_number

{'Apus affinis': 5228662,
 'Apus caffer': 5228644,
 'Parnasius apollo': 1938810,
 'Calotriton arnoldi': 2431827,
 'Gallotia simonyi': 2468673,
 'Alytes dickhilleni': 2426613,
 'Meliataea cinxia': 8168644,
 'Silene ciliata': 7267056,
 'Abies pinsapo': 2685464,
 'Lanius minor': 2492854,
 'Ciconia ciconia': 2481912}

In [7]:
def get_occurences(sp):
    
    years = range(1970, 2020)
    x = []
    
    for y in years:
        data = occ.search(taxonKey = sp, limit = 300, country = 'ES', year = str(y))
        #Get the number of occurrences:
        x.append({str(y): data['count']})
        
        # Set all the data in an unique dictionary
        final = {}
        for d in x:
            for k in d.keys():
                final[k] = final.get(k,0) + d[k]
       
    return final

In [8]:
def get_taxonomic_info(sp):
    data = occ.search(taxonKey = sp, limit = 300, country = 'ES', year = '2016')
    taxonomic = data["results"] 
    for dictionary in taxonomic:
        species = dictionary["scientificName"]
        kingdom = dictionary["kingdom"] 
        genus = dictionary['genus']
        family = dictionary['family']           
        country = dictionary["country"]
        records = dictionary['basisOfRecord']
        pub = dictionary['publishingCountry']
            
        
    
    species_info = {'species': species,
                    'kingdom': kingdom,
                    'Genus': genus, 
                    'Family': family,
                    'country': country,
                    'records': records,
                    'Publishing_country': pub
               }
    return species_info

In [32]:
def join_occurrences_taxonomic(sp):
    '''
    Receive taxonomic information dictionary and dictictionry with occurences/year
    
    Args:
    
    
    Returns
        jsdf
    '''
    # occurrence dictionary to dataframe
    df_occurrences = pd.DataFrame.from_dict(data = get_occurences(sp),  orient='index').reset_index()
    df_occurrences.columns = ['Year','Occurrences']
    
    #taxonomic dictionary to dataframe
    df_taxonomic = pd.DataFrame(data = get_taxonomic_info(sp), index = [0])
    df_taxonomic = pd.concat([df_taxonomic]*50, ignore_index=True)
    
    # concatenate occurences with taxonomic information
    sp1 = pd.concat([df_occurrences,df_taxonomic], axis=1)
    
    
    return sp1

In [10]:
silene = join_occurrences_taxonomic(7267056)
silene[:5]

,Year,Occurrences,species,kingdom,Genus,Family,country,records,Publishing_country
0,1970,6.0,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
1,1971,1.0,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
2,1972,5.0,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
3,1973,9.0,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR
4,1974,6.0,Silene ciliata Pourr.,Plantae,Silene,Caryophyllaceae,Spain,HUMAN_OBSERVATION,FR


In [67]:
def get_all_species_info(code_number, cod_number):
    for key, values in code_number.items():
        data_species = join_occurrences_taxonomic(values)
        return data_species

In [25]:
%%time
test = get_all_species_info(code_number)

CPU times: user 1.03 s, sys: 78.1 ms, total: 1.11 s
Wall time: 37.9 s


In [55]:
test[:5]

,Year,Occurrences,species,kingdom,Genus,Family,country,records,Publishing_country
0,1970,0.0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
1,1971,0.0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
2,1972,0.0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
3,1973,0.0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
4,1974,0.0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES


In [33]:
%time
Apus_affinis = join_occurrences_taxonomic(5228662)
Apus_caffer = join_occurrences_taxonomic(5228644)
Parnasius_apollo = join_occurrences_taxonomic(1938810)
Alytes_dickhilleni = join_occurrences_taxonomic(2426613)
Meliataea_cinxia = join_occurrences_taxonomic(8168644)
Silene_ciliata = join_occurrences_taxonomic(7267056)
Abies_pinsapo = join_occurrences_taxonomic(2685464)
Lanius_minor = join_occurrences_taxonomic(2492854)
Ciconia_ciconia = join_occurrences_taxonomic(2481912)
Sonchus_tenerrimus = join_occurrences_taxonomic(3105768)
Araujia_sericiferas = join_occurrences_taxonomic(3170442)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [62]:
all_species = pd.concat([Apus_affinis, Apus_caffer, Parnasius_apollo,Alytes_dickhilleni,Meliataea_cinxia,Silene_ciliata,
                         Abies_pinsapo,Lanius_minor,Ciconia_ciconia, Sonchus_tenerrimus, Araujia_sericiferas], axis=0)

In [63]:
all_species[:5]

,Year,Occurrences,species,kingdom,Genus,Family,country,records,Publishing_country
0,1970,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
1,1971,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
2,1972,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
3,1973,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES
4,1974,0,"Apus affinis (J.E.Gray, 1830)",Animalia,Apus,Apodidae,Spain,HUMAN_OBSERVATION,ES


In [65]:
all_species.to_csv("species.csv")